In [6]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import keras
from keras.layers import Conv2D, Dropout
from keras.models import Sequential
from keras.models import load_model

from keras.layers import Flatten, Dense
from keras.layers import BatchNormalization
from keras.layers import Conv2D

from keras.optimizers import SGD
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint

from PIL import Image
from pandas import read_csv



train_file = read_csv('train.csv')
train_file=train_file.iloc[:, 1:]
valid_file = read_csv('valid.csv')
valid_file=valid_file.iloc[:, 1:]

img = Image.open('mm/'+ str(25272) + '.png')
img =img.convert('RGB')
width, height = img.size
left = 0
top = height / 6
right = 320
bottom = height
img = img.crop((left, top, right, bottom))
img = img.resize((200,66))
arr = image.img_to_array(img)

batch_size = 128
input_shape=arr.shape
img_shape=input_shape
train_steps = (train_file.shape[0] / batch_size) + 1
val_steps = (valid_file.shape[0] / batch_size) + 1

OUTPUT_NORMALIZATION = 6600

In [7]:
def generator(df, batch_size):
    
        
    img_list = df['nombre']
    wheel_axis = df['steer']    
    # create empty batch
    batch_img = np.zeros((batch_size,) + img_shape)
    batch_label = np.zeros((batch_size, 1))
    
    index = 0
    while True:
        for i in range(batch_size):
            img_name = img_list[index]
            img = Image.open('mm/'+ str(int(img_name)) + '.png')
            img =img.convert('RGB')
            left = 0
            top = height / 6
            right = 320
            bottom = height
            img = img.crop((left, top, right, bottom))
            img = img.resize((200,66))
            arr = image.img_to_array(img)
            
            batch_label[i] = wheel_axis[index]*100 
            batch_img[i] = arr
            
            index += 1
            if index == len(img_list):
                index = 0
            
        yield batch_img / 255.0, (batch_label / OUTPUT_NORMALIZATION)


train_batch = generator(train_file, batch_size)
val_batch = generator(valid_file, batch_size)

        
        
def get_model(input_shape):
    model = Sequential([
        Conv2D(24, kernel_size=(5,5), strides=(2,2), activation='relu', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(36, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(),
        Conv2D(48, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(50, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='relu'),
        BatchNormalization(),
        Dense(1)
    ])
    return model


model = get_model(input_shape)
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss="mse", metrics=['mae', 'acc'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 31, 98, 24)        1824      
_________________________________________________________________
batch_normalization_9 (Batch (None, 31, 98, 24)        96        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
batch_normalization_10 (Batc (None, 14, 47, 36)        144       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 5, 22, 48)         43248     
_________________________________________________________________
batch_normalization_11 (Batc (None, 5, 22, 48)         192       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 3, 20, 64)        

In [8]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

model_file_name= 'drive-model' + '-{epoch:03d}-{val_loss:.5f}.h5'
callbacks_list = [
    ModelCheckpoint(model_file_name, monitor='val_mean_absolute_error', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_mean_absolute_error', patience=3, verbose=0),
    TensorBoard(log_dir='./tensorboard/', histogram_freq=0, write_graph=False, write_images=False)
    ]

In [9]:
model.fit_generator(train_batch, 
                    train_steps, 
                    epochs=10, 
                    verbose=1, 
                    callbacks=callbacks_list, 
                    validation_data=val_batch, 
                    validation_steps=val_steps,
                    initial_epoch=0)

model.save('drive-model.h5')

Epoch 1/10
201/200 [==============================] - 1021s 5s/step - loss: 0.1389 - mae: 0.2321 - acc: 0.1170 - val_loss: 0.0561 - val_mae: 0.1958 - val_acc: 0.1294


C:\Users\Manel\Anaconda3\envs\deeplearning\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_mean_absolute_error available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
C:\Users\Manel\Anaconda3\envs\deeplearning\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,val_mae,val_acc,loss,mae,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 2/10
201/200 [==============================] - 863s 4s/step - loss: 0.0312 - mae: 0.1240 - acc: 0.1260 - val_loss: 0.0541 - val_mae: 0.1503 - val_acc: 0.1299
Epoch 3/10
201/200 [==============================] - 845s 4s/step - loss: 0.0242 - mae: 0.1074 - acc: 0.1265 - val_loss: 0.0253 - val_mae: 0.1020 - val_acc: 0.1297
Epoch 4/10
201/200 [==============================] - 903s 4s/step - loss: 0.0205 - mae: 0.0981 - acc: 0.1265 - val_loss: 0.0306 - val_mae: 0.0917 - val_acc: 0.1294
Epoch 5/10
201/200 [==============================] - 996s 5s/step - loss: 0.0186 - mae: 0.0933 - acc: 0.1264 - val_loss: 0.0179 - val_mae: 0.0883 - val_acc: 0.1305
Epoch 6/10
201/200 [==============================] - 972s 5s/step - loss: 0.0166 - mae: 0.0880 - acc: 0.1267 - val_loss: 0.0118 - val_mae: 0.0830 - val_acc: 0.1296
Epoch 7/10
201/200 [==============================] - 905s 5s/step - loss: 0.0156 - mae: 0.0855 - acc: 0.1265 - val_loss: 0.0165 - val_mae: 0.0810 - val_acc: 0.1297
Epoch 8/10